In [ ]:
#This is a parameters cell.  Any args passed in from the driver notebook will overwrite these defaults

xmla_endpoint = None #None = Current Workspace's XMLA Endpoint
perf_analyzer_filename = '/lakehouse/default/Files/PerfScenarios/Queries/PowerBIPerformanceData.json'
model = "Queries Final Report with RLS"
roles = None  #Used with customdata to force active roles.  Not needed with effective_username
customdata = "foo" #customdata for use in RLS without impersonation
effective_username = None #for RLS with impersionation.  Must be the UPN of a user with read+build for the model
iterations = 3  #number of times to run the perf_analyzer_filename in this scenario
delay_sec = 1  #number of seconds to wait between iterations
loadtestId = "localtesting" # name of load test for logging
threadId = 1 #id of this virtual user to seperate logging for multiuser-testing
concurrent_threads = 1  #number of concurrent threads in this test; passed in here only for logging

In [20]:
# lp = notebookutils.lakehouse.getWithProperties()
# print(lp)

In [ ]:
import pandas
from typing import Iterable
import json
import time
import sempy.fabric as fabric
tom = fabric.create_tom_server() #get CLR loaded
from datetime import datetime


from Microsoft.AnalysisServices.AdomdClient import AdomdConnection

def run_query(con: AdomdConnection, query:str) -> pandas.DataFrame:
    cmd = con.CreateCommand()
    cmd.CommandText = query

    rdr = cmd.ExecuteReader()
    fc = rdr.FieldCount
    
    columns = [rdr.GetName(i) for i in range(fc)]
    
    # print(columns)
    df = pandas.DataFrame(columns=columns)


    rn = 0
    while rdr.Read():
        values = [rdr[i] for i in range(fc)]

        df.loc[rn]=values
        rn = rn + 1       
    
    rdr.Close()
    # display(df)
    return df

def load_queries(fn: str) -> list:
    queries=[]
    # print(f'ppp {fn}')
    # print('--------------------------------')
    # notebookutils.fs.ls('/lakehouse/default/Files/PerfScenarios/Queries/PowerBIPerformanceData.json')
    # print('--------------------------------')

    with open(fn, encoding='utf-8-sig') as f:
        json_string = f.read()
        d = json.loads(json_string)

        visual_name=""
        query_text = ""
        for e in d["events"]:
            

            if e["name"] == "Visual Container Lifecycle":
                visual_name = e["metrics"]["visualTitle"]

            if e["name"] == "Execute DAX Query":
                query_text = e["metrics"]["QueryText"]
                queries.append({"visual_name":visual_name, "query_text":query_text})
    return queries

def run_perf_scenario(con: AdomdConnection, queries: Iterable[str], i: int) -> list:
    results = []
    for qn, q in enumerate(queries, start=1):
        start = time.time()
        query_text = q["query_text"]
        visual_name = q["visual_name"]
        df = run_query(con, query_text)
        duration = time.time() - start
        
        result = {
            "loadtest_id": loadtestId,
            "model": model,
            "concurrent_threads": concurrent_threads,
            "iterations": iterations,
            "delay_sec": delay_sec,
            "query_number": qn,
            "visual_name": visual_name,
            "iteration": i,
            "query": query_text,
            "rows": df.shape[0],
            "duration": duration,
            "start_time": start,
            "start_time_dt": datetime.fromtimestamp(start),
            "customdata": customdata,
            "effective_username": effective_username,
            "thread_id": threadId
        }
        results.append(result)
        time.sleep(delay_sec)
    return results

In [25]:
import sempy.fabric as fabric
import pandas 
import csv
import uuid
from Microsoft.AnalysisServices.AdomdClient import AdomdConnection 

tom = fabric.create_tom_server() #get CLR loaded
token = notebookutils.credentials.getToken("pbi")
notebookutils.fs.mkdirs(f'/lakehouse/default/Files/PerfScenarios/logs/{loadtestId}') #This should already exist, but useful here when running in standalong mode

if (xmla_endpoint == None):
    xmla_endpoint = f"powerbi://api.powerbi.com/v1.0/myorg/{notebookutils.runtime.context['currentWorkspaceName']}"


constr = f"Data Source={xmla_endpoint};Initial Catalog={model};password={token};Timeout=7200;"
if (effective_username != None):
    constr = constr + f"EffectiveUserName={effective_username};"
if (customdata != None):
    constr = constr + f"CustomData={customdata};"
    # constr = constr + f"CustomData=;"
if (roles != None):
     constr = constr + f"Roles={roles};"

print(constr)
con = AdomdConnection(constr)
try:
    con.Open()

    queries = load_queries(perf_analyzer_filename)

    all_results = []
    print(f"starting {iterations} iterations with {delay_sec} think time between")
    for i in range(iterations):
        #print(f"Starting iteration {i}")
        results:list=run_perf_scenario(con,queries,i)
        if (i<iterations):
            print(f"Completed iteration, starting {delay_sec}sec think time")
            time.sleep(delay_sec)
        all_results += results

    
    # Write results to CSV files in lakehouse for analyzing later
    filename = f"{loadtestId}_user_{threadId}"
    with open(f"/lakehouse/default/Files/PerfScenarios/logs/{loadtestId}/{filename}.csv", 'w') as file:
        writer = csv.DictWriter(file, fieldnames=all_results[0].keys())
        writer.writeheader()
        writer.writerows(all_results)

    con.Close()

except Exception as e:
    print(e)
    with open(f"/lakehouse/default/Files/PerfScenarios/logs/{loadtestId}/error.log", 'a') as file:
         file.write(f'{constr}  : {e}\n')
    raise


Data Source=powerbi://api.powerbi.com/v1.0/myorg/Fabric Load Testing;Initial Catalog=DIAD Final Report with RLS;password=[REDACTED];Timeout=7200;CustomData=foo;
ppp /lakehouse/default/Files/PerfScenarios/DIAD/PowerBIPerformanceData.json
--------------------------------
--------------------------------
starting 3 iterations with 1 think time between
Completed iteration, starting 1sec think time
Completed iteration, starting 1sec think time
Completed iteration, starting 1sec think time
